In [1]:
import numpy as np
import menpo.io as mio
#import scipy.io as sio
from pathlib import Path
import os.path
import errno
#import menpo.shape
#%matplotlib inline
import re
from shutil import copyfile
import fnmatch
from PIL import Image
import json
from pprint import pprint
from datetime import datetime

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

<b> Export thumbnails for annotation </b>

In [ ]:
root = '/vol/phoebe/3DMD_SCIENSE_MUSEUM/15_06_2017/'
# root = '/media/ap112/My Passport/22_06_2017/'
dest='/media/ap112/C4184698184688FE/Dropbox/PostDoc/'

num_images=0
num_imgs=[]
size= 640, 480
ilevel=1
num_bmp=0
num_obj=0

# if os.direxists(os.path.join(os.getcwd()), 'new_folder'))
for path, dirs, files in os.walk(root):
    if ilevel==1:
        subjects=dirs
        folder_name=path.split('/')[-2]
        print(path,dirs,files,'folder name is',folder_name)        
    ilevel=ilevel+1
    for name in files:
        if fnmatch.fnmatch(name, '*.bmp'):
            num_images=num_images+1
    if 'meshes' in path.split('/')[-1]:
        for name in files:
#             e = time()
            if fnmatch.fnmatch(name, '*.bmp'):
                
                tmp_name=name.split('/')
#                 old_name=new_name[0]
                new_name=tmp_name[-1].split('.')
                new_path=dest+folder_name+'/'+new_name[0]
#                 print(new_path,'Old name is' ,old_name,new_name)
                if not os.path.exists(new_path):
                    os.makedirs(new_path)
                im = Image.open(path+'/'+name)
                im.thumbnail(size, Image.ANTIALIAS)
                
                im.save(new_path+'/'+new_name[0]+'.'+new_name[1]+'.jpg',quality=90)
                num_bmp=num_bmp+1
#                 print(time() - e)
            if fnmatch.fnmatch(name, '*.obj'):
                num_obj=num_obj+1
#             if old_name != new_name[0]:
#                 print(old_name+' has just finished')


<b> Change json files of a session</b>

In [ ]:
source_path='/vol/atlas/databases/3DMD_SCIENSE_MUSEUM/'
session_name = '04_05_2017'
dest_path_jsons='/vol/phoebe/NEW_3DMD/'
# /vol/atlas has been mapped to Z:
# /vol/phoebe has been mapped to P:
# Relative path for the exporting directory, the path that bmps has been exported
# as it should be written to json file
exportdirectory_path='Z:/databases/3DMD_SCIENSE_MUSEUM'

# Absolute paths for calibrations and mstereo.ini
# destination root path
dest_root_path='P:/NEW_3DMD'
calibations_path=''.join([dest_root_path,'/calibrations'])
mstereo_path=''.join([dest_root_path,'/Mstereo/mstereo_default.ini']) 
#  Absolyte path to back up all the old json files,
#  old json files are also backed up in the same folder changing just the extension
backupjson_path=''.join([dest_path_jsons,session_name,'/backupjson'])


num_images=0
num_imgs=[]
ilevel=1
num_copied=0 

for path, dirs, files in os.walk(''.join([source_path,session_name])):
    if ilevel==1:
        subjects=dirs
#         folder_name=path.split('/')[-1]
        print(path,dirs,files)    
    ilevel+=1
    for name in files:
        if fnmatch.fnmatch(name, '*.json') and len(name.split('.'))==2:
            
            num_images+=1
            source_filename=''.join([path,'/',name])
#             dest_filename=''.join([path,'/',name.split('.')[0],'.bak'])
            dest_filename=''.join([backupjson_path,'/',name])
            if not os.path.exists(os.path.dirname(dest_filename)):
                try:
                    os.makedirs(os.path.dirname(dest_filename))
                except OSError as exc: # Guard against race condition
                    if exc.errno != errno.EEXIST:
                        raise
            print(dest_filename)
#             if os.path.isfile(source_filename):
#                 copyfile(source_filename, dest_filename)
#                 copyfile(source_filename, dest_filename2)
#                 print(source_filename,' copied to ',dest_filename)
#                 print(dest_filename2)
#                 num_copied=num_copied+1
            with open(path+'/'+name) as data_file:    
                data = json.load(data_file)
                data_file.close() # Close the JSON file

#              old format
            exportdirectory=''.join([exportdirectory_path,'/',session_name,'/',name.split('.')[0]]) 
            if os.path.exists(exportdirectory+'/images'):
#             data["exportdirectory"] = ''.join([exportdirectory_path,'/',session_name,'/',name.split('.')[0]]) 
                data["exportdirectory"] = ''.join([exportdirectory,'/images']) 
            else:
                data["exportdirectory"] = exportdirectory 


#             print('exportdirectory: ',data["exportdirectory"])
            data["exportmstereoini"]=mstereo_path
#             print('exportmstereoini: ',data["exportmstereoini"])
            data["exporttkadir"]=''.join([calibations_path,'/', data["exporttkadir"].split('/')[-1]])
#             print('exporttkadir: ',data["exporttkadir"])
            data["exportmeshdir"]=''.join([dest_root_path,'/',session_name,'/',name.split('.')[0],'/meshes'])  #'P:/NEW_3DMD/10_04_2017/00002/"
            data.pop('exportimagedir', None)
#             print('exportmeshdir: ',data["exportmeshdir"])
#             print(data)
## Save our changes to JSON file
#             if name=='00001.json':
#                   print('00001.json')
            with open(dest_filename, 'w') as outfile:  
                json.dump(data, outfile)
                outfile.close()

In [ ]:
with open('/vol/atlas/databases/3DMD_SCIENSE_MUSEUM/11_05_2017/02075/02075.json') as data_file:
     data = json.load(data_file)
     data_file.close()

<b>Functions used to handle Google Spreadsheet</b>

In [3]:
# use creds to create a client to interact with the Google Drive API
def read_google_sheets(spreadsheet_name):
    scope = ['https://spreadsheets.google.com/feeds']
    creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
    client = gspread.authorize(creds)
 
    # Find a workbook by name and open the first sheet
    # Make sure you use the right name here.
    sheet = client.open(spreadsheet_name).sheet1
 
    # Extract and print all of the values
    list_of_hashes = sheet.get_all_records()
    list_of_hashes=list(filter(lambda x: x['Id']!='', list_of_hashes))
    # a = [x for x in new_list if x['Id'] != 250]
    return list_of_hashes

def searchdate(search_year, people):
        return [element for element in people if datetime.strptime(element['DateBirth'],'%d/%m/%Y').year<search_year and element['Ethnicity']=='White' ]

def searchidn(idn, people):
    return [element for element in people if element['Gender'] == idn]

def searchnogender(people):
    return [element for element in people if element['Gender'] != 'Male' and element['Gender'] != 'Female']


def copy_obj_bmp2jpg_files(root_path,dest_path,list_of_people):
    exist_lists=[]
    num_copied_subj=0
    num_not_found_subj=0
    not_found_list=[]
    if not os.path.exists(os.path.dirname(dest_path)):
        try:
            os.makedirs(os.path.dirname(dest_path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    for person in list_of_people:
    #Transform the date to the one that is used for folders, namely dd_mm_yyyy
        if person['Id']<250:
            dt=datetime.strptime(person['Timestamp'],'%d/%m/%Y')
        else:
            dt=datetime.strptime(person['Timestamp'],'%d/%m/%Y %X')
        if dt.day<10:
            tmpday=''.join(['0',str(dt.day)])
        else:
            tmpday=str(dt.day)
        date_session=''.join([tmpday,'_0',str(dt.month),'_',str(dt.year)])
        #Check if  there is zero padding
        zeroes_padding=num_zeroes_in_filename(''.join([root_path,date_session]),str(person['Id']))
        if zeroes_padding==-1: 
            print(''.join([date_session,'/',str(person['Id'])]),'cannot be found')
            num_not_found_subj+=1
            not_found_list.append(person['Id'])
            continue
        
        fileid=str(person['Id']).rjust(zeroes_padding,'0')
        source_path=''.join([root_path,date_session,'/',fileid,'/meshes/'])
        if os.path.exists(os.path.dirname(source_path)):
            filename=''.join([fileid,'.000001'])   
            bmpfile=''.join([filename,'.bmp'])
            objfile=''.join([filename,'.obj'])
            if os.path.isfile(source_path+bmpfile):
                          print(source_path+bmpfile,source_path+objfile,' are copying')
                          num_copied_subj+=1
                          copyfile(source_path+objfile, dest_path+objfile)
                          im = Image.open(source_path+bmpfile)
                          im.save(dest_path+filename+'.jpg',quality=90)    
            exist_lists.append(source_path)
    return num_not_found_subj,num_copied_subj,not_found_list

def num_zeroes_in_filename(file_path,person_id):
    len_id=len(person_id)
    if  os.path.exists(''.join([file_path,'/',person_id])):
            return len_id
    elif  os.path.exists(''.join([file_path,'/0',person_id])):
            return len_id+1
    elif  os.path.exists(''.join([file_path,'/00',person_id])):
            return len_id+2
    elif  os.path.exists(''.join([file_path,'/000',person_id])):
            return len_id+3
    elif  os.path.exists(''.join([file_path,'/0000',person_id])):
            return len_id+4
    else: 
            return -1
def printonekey(printedkeys,people):
    for person in people:
        for prkey in printedkeys:
            print(person[prkey])

In [5]:
spreadsheet_name= "Copy Museum data"

list_of_hashes=read_google_sheets(spreadsheet_name)

In [ ]:
nogender=searchnogender(list_of_hashes)
print(len(nogender))
printonekey(['Id','Gender','Email'],nogender)

In [6]:
males=searchidn('Male', list_of_hashes)
females=searchidn('Female', list_of_hashes)
no_gender=searchidn('Other', list_of_hashes)
print('There are',len(males),'males', len(females),'females and',len(no_gender),'others')
print('Total: ',len(no_gender)+len(males)+len(females))


There are 2325 males 2597 females and 9 others
Total:  4931


In [127]:
print(list_of_hashes[205])

{'Mothers Country': '', 'Fathers Ethnicity': 'White', 'Mothers ethnicity': 'White', 'Id': 206, 'Gender': 'Female', 'Email': 'tomburstow@hotmail.com', 'Country': 'United Kingdom', 'Fathers Country': '', 'Timestamp': '12/04/2017', 'DateBirth': '12/07/2007', 'Language': '', 'Ethnicity': 'White', '': ''}


In [ ]:
(not_found, copied) = copy_obj_bmp2jpg_files(root_path,dest_path,above60white)

In [116]:
print(len(list_of_hashes))
i=1
tmp_num=list_of_hashes[0]['Id']
tmp_list=list_of_hashes[1:]
for person in tmp_list:
    if  person['Id']-tmp_num!=1:
        print( person['Id'])
        i+=1
    tmp_num=person['Id']

4931
209
469
704
760
1453
1450
1454
1619
1744
1741
1745


<p><b> Extract obj and jpg files from specific people</b></p>
<b> Number 207, 208, 468,703, 759, 1617, 1618 are missing from Google Spreadsheets.</b>

In [ ]:
# root_path='/vol/phoebe/3DMD_SCIENCE_MUSEUM/'
root_path='/vol/phoebe/NEW_3DMD/'
# root_path='/vol/atlas/databases/3DMD_SCIENSE_MUSEUM/'
dest_path='/data/Dropbox/Dropbox/Dropbox/females/'
# spreadsheet_name= "Copy Museum data"
#
# list_of_hashes=read_google_sheets(spreadsheet_name)
print('List has been read')
# above60white=searchdate(1967,list_of_hashes)
# num_not_found, num_copied = copy_obj_bmp2jpg_files(root_path,dest_path,above60white)

# nogender=searchnogender(list_of_hashes)
num_not_found, num_copied,list_notfound = copy_obj_bmp2jpg_files(root_path,dest_path,females)
print(num_copied,' were copied')

<b>Export thumbnails by 4G<b>

In [ ]:
import menpo.io as mio

from menpo.visualize import print_progress
from pathlib import Path

def process_image(im):
    im = im.rescale_to_diagonal(700)
    subject_id = im.path.stem.split('.')[0]
    path =  Path('tmp') / subject_id / im.path.with_suffix('.jpg').name
    
    try:
        mio.export_image(im, path, overwrite=True)
    except:
        path.parent.mkdir(exist_ok=True, parents=True)
        mio.export_image(im, path)

# for im in print_progress(mio.import_images('/media/ap112/Seagate Expansion Drive/11_06_2017/**/meshes/*.bmp')):


In [ ]:
from multiprocessing import Pool

pool = Pool(4)

pool.map_async(process_image, mio.import_images('/media/ap112/Seagate Expansion Drive/11_06_2017/**/meshes/*.bmp'))

In [ ]:
pool.map_async()